In [1]:
from sys import platform
if platform == 'linux' or platform == 'linux2':
    datapath = '/home/alin/MyLearning/Kaggle/Sberbank/data/'
else:
    datapath = 'C:/Users/alin/Documents/SelfStudy/MyLearning/Kaggle/Sberbank/data/'
print datapath

/home/alin/MyLearning/Kaggle/Sberbank/data/


In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

In [3]:
import xgboost as xgb

In [4]:
from xgboost.sklearn import XGBClassifier

In [5]:
%matplotlib inline

In [6]:
# From here: https://www.kaggle.com/robertoruiz/sberbank-russian-housing-market/dealing-with-multicollinearity/notebook
macro_cols = ["balance_trade", "balance_trade_growth", "eurrub", "average_provision_of_build_contract",
"micex_rgbi_tr", "micex_cbi_tr", "deposits_rate", "mortgage_value", "mortgage_rate",
"income_per_cap", "rent_price_4+room_bus", "museum_visitis_per_100_cap", "apartment_build"]

In [7]:
%qtconsole

In [8]:
train_df = pd.read_csv(datapath + 'train.csv', parse_dates=['timestamp'])
macro_df = pd.read_csv(datapath + 'macro.csv', parse_dates=['timestamp'], usecols=['timestamp'] + macro_cols)


In [9]:
ylog_train_all = np.log1p(train_df['price_doc'].values)


In [10]:
train_df.drop(['id', 'price_doc'], axis = 1, inplace = True)

In [11]:
all_df = pd.merge(train_df, macro_df, on = 'timestamp', how = 'left')

In [12]:
month_year = (all_df.timestamp.dt.month + all_df.timestamp.dt.year * 100)
month_year_cnt_map = month_year.value_counts().to_dict()
all_df['month_year_cnt'] = month_year.map(month_year_cnt_map)

In [13]:
week_year = (all_df.timestamp.dt.weekofyear + all_df.timestamp.dt.year * 100)
week_year_cnt_map = week_year.value_counts().to_dict()
all_df['week_year_cnt'] = week_year.map(week_year_cnt_map)

In [14]:
all_df['month'] = all_df.timestamp.dt.month
all_df['dow'] = all_df.timestamp.dt.dayofweek

In [15]:
all_df['rel_floor'] = all_df['floor'] / all_df['max_floor'].astype(float)

In [16]:
all_df['rel_kitch_sq'] = all_df['kitch_sq'] / all_df['full_sq'].astype(float)

In [17]:
all_df.drop(['timestamp'], axis = 1, inplace = True)

In [18]:
num_df = all_df.select_dtypes(include = [np.number])
cat_df = all_df.select_dtypes(exclude = [np.number]).copy()

In [19]:
for c in cat_df:
    cat_df[c] = pd.factorize(cat_df[c])[0]

In [20]:
df_values = pd.concat([num_df, cat_df], axis = 1)

In [21]:
X_all = df_values.values

In [22]:
num_val = int(X_all.shape[0] * 0.2)
num_train = X_all.shape[0] - num_val

In [23]:
X_train = X_all[:num_train]

X_valid = X_all[num_train:]

ylog_train = ylog_train_all[:num_train]

ylog_valid = ylog_train_all[num_train:]

In [24]:
num_train0 = int(num_train * 0.2)
X_train0 = X_train[:num_train0]
X_valid0 = X_train[num_train0:]
ylog_train0 = ylog_train[:num_train0]
ylog_valid0 = ylog_train[num_train0:]

In [25]:
df_columns = df_values.columns

In [26]:
dtrain = xgb.DMatrix(X_train, ylog_train, feature_names=df_columns)

In [27]:
dval = xgb.DMatrix(X_valid, feature_names=df_columns)

In [28]:
dtrain0 = xgb.DMatrix(X_train0, ylog_train0, feature_names= df_columns)
dvalid0 = xgb.DMatrix(X_valid0, ylog_valid0, feature_names= df_columns)

In [29]:
def rmsle(lp, la):
    return np.sqrt(np.mean(np.square(lp - la)))

In [47]:
def modelfit(alg, Xt, yt, useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(Xt, yt)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='rmse', early_stopping_rounds=early_stopping_rounds, verbose_eval=False)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    print 'opt iterations = ', cvresult.shape[0]
    alg.fit(Xt, yt ,eval_metric='rmse')
        
    #Predict training set:
    dtrain_predictions = alg.predict(Xt)
    
        
    #Print model report:
    print "\nModel Report"
    print "rmse : %.4g" % metrics.rmse_score(dtrain_predictions, yt)
                    
    #feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
    #feat_imp.plot(kind='bar', title='Feature Importances')
    #plt.ylabel('Feature Importance Score')

In [31]:
xgb1 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=115,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'reg:linear',
 silent = 0,
 seed=27)
#modelfit(xgb1, X_train, ylog_train )

In [31]:
params = {
    'objective': 'reg:linear',
    'max_depth': 5,
    'learning_rate': 0.1,
    'silent': 1.0,
    'n_estimators': 2,
    'min_child_weight': 1,
    'gamma': 0,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'seed': 27
}

In [ ]:
bst = XGBClassifier(**params).fit(X_train, ylog_train, verbose = True)

In [ ]:
xgb1.fit(X_train, ylog_train, eval_metric = 'rmse')

In [ ]:
xgb_params = {
    'eta': 0.05,
    'max_depth': 5,
    'subsample': 1.0,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 1
}

partial_model = xgb.train(xgb_params, dtrain, num_boost_round=1000, evals=[(dval, 'val')],
                       early_stopping_rounds=20, verbose_eval=20)


In [128]:
xgb_params1 = { 'eta': 0.1, 'max_depth': 3, 'subsample': 0.8,  'eval_metric': 'rmse'}

In [120]:
model0 = xgb.train(xgb_params, dtrain0, num_boost_round = 1000, evals=[(dvalid0, 'val0')], 
                   early_stopping_rounds = 20, verbose_eval = 20)

[0]	val0-rmse:14.3729
Will train until val0-rmse hasn't improved in 20 rounds.
[20]	val0-rmse:5.2655
[40]	val0-rmse:2.0529
[60]	val0-rmse:0.957311
[80]	val0-rmse:0.627749
[100]	val0-rmse:0.544778
[120]	val0-rmse:0.523233
[140]	val0-rmse:0.518817
[160]	val0-rmse:0.517428
Stopping. Best iteration:
[158]	val0-rmse:0.517222



In [125]:
model1 = xgb.train(xgb_params1, dtrain, num_boost_round = 300)

In [129]:
num_rounds = 300 # how many estimators
hist = xgb.cv(xgb_params1, dtrain, num_rounds, nfold=5, metrics={'rmse'}, 
              early_stopping_rounds = 30, verbose_eval = 30, seed = 1983)
#hist

[0]	train-rmse:13.5867+0.0013655	test-rmse:13.5867+0.00668001
[30]	train-rmse:0.752034+0.00165326	test-rmse:0.755884+0.0032213
[60]	train-rmse:0.468277+0.00317942	test-rmse:0.480555+0.0118964
[90]	train-rmse:0.458668+0.00326311	test-rmse:0.476454+0.0119979
[120]	train-rmse:0.45154+0.00320829	test-rmse:0.474773+0.0118628
[150]	train-rmse:0.445083+0.00317647	test-rmse:0.474134+0.011661
[180]	train-rmse:0.439518+0.00298479	test-rmse:0.474012+0.0116063
[210]	train-rmse:0.434416+0.00319044	test-rmse:0.473796+0.0115415
[240]	train-rmse:0.429502+0.00309017	test-rmse:0.473665+0.0115035


In [123]:

#watchlist = [(dtrain, 'train')]

#model1 = xgb.train(xgb_params1, dtrain, num_boost_round = 1000, evals = watchlist, early_stopping_rounds = 20,
#                  verbose_eval = 20)

In [126]:
pred_train = model1.predict(dtrain)
pred_valid = model1.predict(dval)
print rmsle(pred_train, ylog_train)
print rmsle(pred_valid, ylog_valid)

0.308468130457
0.440355450041


0.150935053058


In [113]:
print rmsle(pred_valid, ylog_valid)

0.475545644267


In [107]:
pred_train2 = partial_model.predict(dtrain)
pred_valid2 = partial_model.predict(dval)

In [108]:
print rmsle(pred_train2, ylog_train)
print rmsle(pred_valid2, ylog_valid)

0.418201616459
0.421250702114
